## LSA
TF-IDF 행렬은 단어의 의미를 전혀 고려하지 못함   
LSA는 기본적으로 DTM(문서 단어 행렬 : 다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현)이나 TF-IDF 행렬에 절단된 SVD를 사용하여 차원을 축소시키고, 단어들의 잠재적인 의미를 끌어냄

In [1]:
import numpy as np

### DTM 만들기

In [2]:
A = np.array([[0,0,0,1,0,1,1,0,0],[0,0,0,1,1,0,1,0,0],[0,1,1,0,2,0,0,0,0],[1,0,0,0,0,0,0,1,1]])
print('DTM의 크기 : ', np.shape(A))

DTM의 크기 :  (4, 9)


### full SVD

In [4]:
# U : 행렬 / s : 대각 행렬 / VT : V의 전치 행렬
U, s, VT = np.linalg.svd(A, full_matrices = True)

print('행렬 U : ')
print(U.round(2))
print('행렬 U의 크기(shape) : ', np.shape(U))

행렬 U : 
[[-0.24  0.75  0.   -0.62]
 [-0.51  0.44 -0.    0.74]
 [-0.83 -0.49 -0.   -0.27]
 [-0.   -0.    1.    0.  ]]
행렬 U의 크기(shape) :  (4, 4)


In [5]:
print('특이값 벡터 : ')
print(s.round(2))
print('특이값 벡터의 크기(shape) : ', np.shape(s))

특이값 벡터 : 
[2.69 2.05 1.73 0.77]
특이값 벡터의 크기(shape) :  (4,)


In [6]:
# 대각 행렬의 크기인 4x9의 임의의 행렬 생성
S = np.zeros((4,9))

# 특이값을 대각행렬에 삽입
S[:4, :4] = np.diag(s)

print('대각 행렬 S : ')
print(S.round(2))

print('대각 행렬의 크기(shape) : ')
print(np.shape(S))

대각 행렬 S : 
[[2.69 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   2.05 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.73 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.77 0.   0.   0.   0.   0.  ]]
대각 행렬의 크기(shape) : 
(4, 9)


In [7]:
print('직교행렬 VT : ')
print(VT.round(2))

print('직교 행렬 VT의 크기(shape) : ')
print(np.shape(VT))

직교행렬 VT : 
[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]
 [ 0.58 -0.    0.    0.   -0.    0.   -0.    0.58  0.58]
 [ 0.   -0.35 -0.35  0.16  0.25 -0.8   0.16 -0.   -0.  ]
 [-0.   -0.78 -0.01 -0.2   0.4   0.4  -0.2   0.    0.  ]
 [-0.29  0.31 -0.78 -0.24  0.23  0.23  0.01  0.14  0.14]
 [-0.29 -0.1   0.26 -0.59 -0.08 -0.08  0.66  0.14  0.14]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19  0.75 -0.25]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19 -0.25  0.75]]
직교 행렬 VT의 크기(shape) : 
(9, 9)


In [8]:
# U * S * VT = A 확인
np.allclose(A, np.dot(np.dot(U,S), VT).round(2))

True

### 절단된 SVD

In [9]:
# 특이값 상위 2개만 보존
S = S[:2, :2]

print('대각 행렬 S : ')
print(S.round(2))

대각 행렬 S : 
[[2.69 0.  ]
 [0.   2.05]]


In [13]:
U = U[:, :2]

print('행렬 U : ')
print(U.round(2))
print(np.shape(U))

행렬 U : 
[[-0.24  0.75]
 [-0.51  0.44]
 [-0.83 -0.49]
 [-0.   -0.  ]]
(4, 2)


축소된 U는 4x2의 크기를 가짐   
이는 문서의 개수 x 토픽의 수 t의 크기   
단어의 개수인 9는 유지되지 않는데 문서의 개수인 4의 크기가 유지됨   
=> 4개의 문서 각각을 2개의 값으로 표현   
   
=>> U의 각 행은 잠재 의미를 표현하기 위한 수치화 된 각각의 문서 벡터


In [11]:
VT = VT[:2, :]

print('직교행렬 VT : ')
print(VT.round(2))

직교행렬 VT : 
[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]]


축소된 VT는 2x9의 크기를 가짐   
이는 토픽의 수 t x 단어의 개수의 크기   
   
=>> VT의 각 열은 잠재 의미를 표현하기 위해 수치화된 각각의 단어 벡터

In [12]:
A_prime = np.dot(np.dot(U,S), VT)

print(A)
print(A_prime.round(2))

[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
[[ 0.   -0.17 -0.17  1.08  0.12  0.62  1.08 -0.   -0.  ]
 [ 0.    0.2   0.2   0.91  0.86  0.45  0.91  0.    0.  ]
 [ 0.    0.93  0.93  0.03  2.05 -0.17  0.03  0.    0.  ]
 [ 0.    0.    0.    0.    0.   -0.    0.    0.    0.  ]]
